In [1]:
#!pip install chesslab

In [2]:
import numpy as np
from chesslab.utils import load_pkl
from chesslab.training_tf import fitting
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:

lr = 0.1
epochs=20
batch_size = 128
test_percent=0.1

path = 'D:/database/ccrl/'
name_data='ccrl_states_elo3.pkl'
name_labels='ccrl_results_elo3.pkl'
save_name='./tmp/tf_weights-relu-elo3.3'

optim = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=lr)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [4]:
download=False
if download:
    from chesslab.utils import download_7z
    path='./'
    file_id = '1MFHFz_rxNziYSeN-9ruwnRiYskd0_9ss'
    download_7z(file_id,path)

In [5]:
encoding_4={
    '.':np.array([0],dtype=np.float32),
    'p':np.array([1/12],dtype=np.float32),
    'P':np.array([2/12],dtype=np.float32),
    'b':np.array([3/12],dtype=np.float32),
    'B':np.array([4/12],dtype=np.float32),
    'n':np.array([5/12],dtype=np.float32),
    'N':np.array([6/12],dtype=np.float32),
    'r':np.array([7/12],dtype=np.float32),
    'R':np.array([8/12],dtype=np.float32),
    'q':np.array([9/12],dtype=np.float32),
    'Q':np.array([10/12],dtype=np.float32),
    'k':np.array([11/12],dtype=np.float32),
    'K':np.array([12/12],dtype=np.float32)
}

In [6]:
class Model_4():

    def __init__(self,
                   n_classes=2):
        initializer = tf.keras.initializers.GlorotNormal()
        self.hw=[]
        self.hb=[]

        self.hw.append( tf.Variable(initializer(shape=(7,7,1,32),dtype=np.float32),name="hl1weigths",dtype="float32") )
        self.hb.append( tf.Variable(np.zeros(32,dtype=np.float32),name="hl1bias",dtype="float32") )
        #8x8x32
        self.hw.append( tf.Variable(initializer(shape=(5,5,32,64),dtype=np.float32),name="hl2weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(64,dtype=np.float32),name="hl2bias",dtype="float32"))
        #8x8x64
        self.hw.append( tf.Variable(initializer(shape=(3,3,64,128),dtype=np.float32),name="hl3weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(128,dtype=np.float32),name="hl3bias",dtype="float32"))
        #8x8x128
        self.hw.append( tf.Variable(initializer(shape=(8*8*128,256),dtype=np.float32),name="hl4weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(256,dtype=np.float32),name="hl4bias",dtype="float32"))

        self.hw.append( tf.Variable(initializer(shape=(256, n_classes),dtype=np.float32),name="outweigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(n_classes,dtype=np.float32),name="outbias",dtype="float32"))

        self.trainable_variables = []
        for i in range(len(self.hw)):
            self.trainable_variables.append(self.hw[i])    
            self.trainable_variables.append(self.hb[i])

    def __call__(self,x): 

        out = tf.cast(x, tf.float32)
        out = tf.reshape(out, shape=[-1, 8, 8, 1])

        layer=0
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)
              #8*8*32
        layer+=1
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)
              #8*8*64
        layer+=1  
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.elu(out)
              #8*8*128
        layer+=1
        out =  tf.reshape(out,[-1, 8*8*128])
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)

        layer+=1
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])

        return out

In [7]:
np.random.seed(0)
tf.random.set_seed(0)

x_data = load_pkl(path+name_data)
y_data = load_pkl(path+name_labels)[:,1] #Nota: pasa de onehot a logits

print(x_data.shape)
print(y_data.shape)

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size = test_percent, random_state = 0, shuffle = True)

del x_data
del y_data

(2661290, 64)
(2661290,)


In [8]:

model = Model_4()
encoding=encoding_4


In [ ]:
fitting(epochs=epochs,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        model=model,
        optimizer=optim,
        batch_size=batch_size,
        lr=lr,
        loss_fn=loss_fn,
        save_name=save_name,
        encoding=encoding)

2021-10-29 23:07:35


Epoch: 01/20 | time: 106s = 1.8m | train loss: 0.6217 | train acc: 0.6523 | test loss: 0.6141 | test acc: 0.6592

Epoch: 02/20 | time: 100s = 1.7m | train loss: 0.5832 | train acc: 0.6891 | test loss: 0.5637 | test acc: 0.7055

Epoch: 03/20 | time: 100s = 1.7m | train loss: 0.5283 | train acc: 0.7318 | test loss: 0.5034 | test acc: 0.7496

Epoch: 04/20 | time: 114s = 1.9m | train loss: 0.4434 | train acc: 0.7872 | test loss: 0.4391 | test acc: 0.7933

...

Epoch: 07/20 | time: 101s = 1.7m | train loss: 0.2390 | train acc: 0.8962 | test loss: 0.3202 | test acc: 0.8655

Epoch: 08/20 | time: 100s = 1.7m | train loss: 0.2007 | train acc: 0.9142 | test loss: 0.3052 | test acc: 0.8769

Epoch: 09/20 | time: 101s = 1.7m | train loss: 0.1724 | train acc: 0.9273 | test loss: 0.3329 | test acc: 0.8747

Epoch: 10/20 | time: 100s = 1.7m | train loss: 0.1497 | train acc: 0.9374 | test loss: 0.2719 | test acc: 0.8979

Epoch: 11/20 | train progress: 23/100 | train loss:0.1392 | train acc: 0.9419           

In [9]:
fitting(epochs=20,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        model= model, 
        load_name=save_name+'.10.h5',
        save_name=save_name,)

2021-10-30 00:05:59


Epoch: 11/30 | time: 103s = 1.7m | train loss: 0.1323 | train acc: 0.9453 | test loss: 0.2550 | test acc: 0.9051

Epoch: 12/30 | time: 109s = 1.8m | train loss: 0.1186 | train acc: 0.9512 | test loss: 0.2881 | test acc: 0.9006

Epoch: 13/30 | time: 100s = 1.7m | train loss: 0.1072 | train acc: 0.9563 | test loss: 0.2610 | test acc: 0.9101

Epoch: 14/30 | time: 101s = 1.7m | train loss: 0.0983 | train acc: 0.9603 | test loss: 0.2676 | test acc: 0.9080

...

Epoch: 26/30 | time: 102s = 1.7m | train loss: 0.0564 | train acc: 0.9792 | test loss: 0.2610 | test acc: 0.9311

Epoch: 27/30 | time: 101s = 1.7m | train loss: 0.0551 | train acc: 0.9797 | test loss: 0.2520 | test acc: 0.9336

Epoch: 28/30 | time: 100s = 1.7m | train loss: 0.0547 | train acc: 0.9801 | test loss: 0.4208 | test acc: 0.9050

Epoch: 29/30 | time: 99s = 1.7m | train loss: 0.0527 | train acc: 0.9809 | test loss: 0.2923 | test acc: 0.9282

Epoch: 30/30 | time: 100s = 1.7m | train loss: 0.0524 | train acc: 0.9811 | test loss: 0.2559 | test acc: 0.9353

In [10]:
fitting(epochs=10,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        model= model, 
        load_name=save_name+'.30.h5',
        save_name=save_name,)

2021-10-30 00:40:22


Epoch: 31/40 | time: 101s = 1.7m | train loss: 0.0516 | train acc: 0.9815 | test loss: 0.2812 | test acc: 0.9336

Epoch: 32/40 | time: 101s = 1.7m | train loss: 0.0513 | train acc: 0.9818 | test loss: 0.2887 | test acc: 0.9329

Epoch: 33/40 | time: 104s = 1.7m | train loss: 0.0501 | train acc: 0.9824 | test loss: 0.3058 | test acc: 0.9294

Epoch: 34/40 | time: 101s = 1.7m | train loss: 0.0498 | train acc: 0.9825 | test loss: 0.2611 | test acc: 0.9374

...

Epoch: 36/40 | time: 101s = 1.7m | train loss: 0.0494 | train acc: 0.9830 | test loss: 0.2928 | test acc: 0.9346

Epoch: 37/40 | time: 102s = 1.7m | train loss: 0.0491 | train acc: 0.9832 | test loss: 0.5695 | test acc: 0.8957

Epoch: 38/40 | time: 100s = 1.7m | train loss: 0.0491 | train acc: 0.9833 | test loss: 0.2913 | test acc: 0.9358

Epoch: 39/40 | time: 101s = 1.7m | train loss: 0.0487 | train acc: 0.9835 | test loss: 0.2947 | test acc: 0.9360

Epoch: 40/40 | time: 101s = 1.7m | train loss: 0.0496 | train acc: 0.9833 | test loss: 0.2934 | test acc: 0.9354